<a href="https://colab.research.google.com/github/aderdouri/ql_web_app/blob/master/tf_quant_finance_notebooks/longstaff_schwartz_least_square_mc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Values Amercian style options using the LSM algorithm.


In [1]:
import os
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'

In [2]:
!pip install tf-quant-finance

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 10.9 MB/s eta 0:00:00


In [3]:
import numpy as np
import tf_quant_finance as tff
import tensorflow as tf

from tf_quant_finance.experimental.lsm_algorithm import make_basket_put_payoff
from tf_quant_finance.experimental.lsm_algorithm import make_polynomial_basis
from tf_quant_finance.experimental.lsm_algorithm import least_square_mc

## tf_quant_finance.models.longstaff_schwartz.make_polynomial_basis

In [6]:
basis = tff.experimental.lsm_algorithm.make_polynomial_basis_v2(2)
x = [[1.0], [2.0], [3.0], [4.0]]
x = tf.expand_dims(x, axis=-1)
basis(x, tf.constant(0, dtype=np.int32))
# Expected result:
[[ 1.  ,  1.  ,  1.  ,  1.  ], [-1.5 , -0.5 ,  0.5 ,  1.5 ],
[ 2.25,  0.25,  0.25,  2.25]]

[[1.0, 1.0, 1.0, 1.0], [-1.5, -0.5, 0.5, 1.5], [2.25, 0.25, 0.25, 2.25]]

In [35]:
# Black-Scholes Parameters
S0 = 100.0  # Initial stock price
sigma = 0.25  # Volatility
rate = 0.05  # Risk-free rate
expiry = 180/365  # Time to maturity (in years)
strike = 100.0  # Strike price
num_samples = 100000  # Number of Monte Carlo paths
time_steps = 50  # Number of time steps for Euler scheme

# Drift and Volatility Functions
def drift_fn(_, x):
    return tf.ones_like(x, dtype=tf.float64) * (rate - 0.5 * sigma**2)

def vol_fn(_, x):
    return tf.expand_dims(tf.ones_like(x, dtype=tf.float64) * sigma, axis=-1)

# Euler Scheme to Simulate Log Stock Price Paths
times = np.linspace(0.0, expiry, num=time_steps, dtype=np.float64)

log_paths = tff.models.euler_sampling.sample(
    dim=1,
    drift_fn=drift_fn,
    volatility_fn=vol_fn,
    random_type=tff.math.random.RandomType.PSEUDO_ANTITHETIC,
    times=times,
    num_samples=num_samples,
    seed=42,
    time_step=expiry / time_steps,
    dtype=tf.float64,
)

# Compute Stock Prices (Exponentiate Log Paths)
paths = tf.exp(log_paths) * S0  # Convert log(S_t) to S_t
# Define variables for American option pricing
exercise_times = tf.range(times.shape[-1])

discount_factors = tf.exp(-rate * tf.constant(times, dtype=tf.float64))  # Discount factor

# Use the built-in payoff and basis functions
payoff_fn = make_basket_put_payoff(strike, dtype=tf.float64)  # Built-in payoff function
basis_fn = make_polynomial_basis(10)  # Built-in polynomial basis function

# Compute American Option Price
american_option_price = least_square_mc(
    paths,
    exercise_times,
    payoff_fn,
    basis_fn,
    discount_factors=discount_factors,
)

# European Put Option Pricing using Black-Scholes Formula
european_option_price = tff.black_scholes.option_price(
    volatilities=[sigma],
    strikes=[strike],
    expiries=[expiry],
    spots=[S0],
    discount_factors=[discount_factors[-1]],
    is_call_options=False,
    dtype=tf.float64,
)

# Display Results
print(f"American Put Option Price: {american_option_price.numpy()}")
print(f"European Put Option Price: {european_option_price.numpy()}")

American Put Option Price: [1.61276118]
European Put Option Price: [5.75978989]


In [42]:
import numpy as np
import pandas as pd

def longstaff_schwartz(paths, strike, discount_rate, exercise_dates):
    """
    Longstaff-Schwartz algorithm for American option pricing.

    Args:
        paths: A NumPy array of shape (number_of_paths, number_of_time_steps)
        representing the simulated asset price paths.
        strike: The strike price for the option. # Changed from strikes to strike
        discount_rate: The risk-free discount rate.
        exercise_dates: A list or array of exercise dates (timesteps).

    Returns:
        The estimated price of the American option.
    """

    n_paths = paths.shape[0]
    n_steps = paths.shape[1]

    # Initialize the continuation values and cashflows
    continuation_values = np.zeros(n_paths)
    cashflows = np.zeros(n_paths)

    # Iterate backward through the time steps
    for t in reversed(range(0, n_steps - 1)): # Iterate up to second to last timestep
        # Determine intrinsic value of option
        intrinsic_value = np.maximum(0, paths[:, t + 1] - strike) # Changed paths[t + 1] to paths[:, t + 1] and strikes to strike

        # Fit a regression model to estimate continuation value
        if np.any(intrinsic_value): # Only perform regression if there are positive intrinsic values
          regression_data = pd.DataFrame({'Price': paths[:, t], 'IntrinsicValue': intrinsic_value}) # Changed paths[t] to paths[:, t]
          regression = np.polyfit(regression_data['Price'], regression_data['IntrinsicValue'], 3)
          continuation_values = np.polyval(regression, paths[:, t]) # Changed paths[t] to paths[:, t]
        else:
            continuation_values = np.zeros(n_paths) # if intrinsic is always zero

        # Compare intrinsic value to continuation value, updating cash flows
        exercise = intrinsic_value > continuation_values
        cashflows[exercise] = intrinsic_value[exercise]

        # Discount cash flows to previous time step
        continuation_values = np.where(exercise, 0, continuation_values) # update value of continuation values

        # discount all values at each timestep
        continuation_values = continuation_values / (1+discount_rate)


        # Update cash flows
        cashflows = np.where(exercise, intrinsic_value, cashflows)

    return np.mean(cashflows)

# Example Usage (replace with your data)
np.random.seed(0)
n_paths = 100000
n_steps = 100
paths = np.random.randn(n_paths, n_steps).cumsum(axis=1)
strike = 1.1  # Example strike price # Changed strikes to strike

discount_rate = 0.05
exercise_dates = list(range(0, n_steps))

option_price = longstaff_schwartz(paths, strike, discount_rate, exercise_dates) # Changed strikes to strike
print(f"Estimated American option price: {option_price}")

Estimated American option price: 0.39459103221246034
